In [126]:
import numpy as np
from qiskit_aer import Aer
from qiskit.circuit.library import QFT, PhaseEstimation, UnitaryGate
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from scipy.linalg import expm
from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal
from qiskit.visualization import plot_histogram
from qiskit_aer.primitives import SamplerV2
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_state_city


class HHL():
    r"""Systems of linear equations arise naturally in many real-life applications in a wide range
    of areas, such as in the solution of Partial Differential Equations, the calibration of
    financial models, fluid simulation or numerical field calculation. The problem can be defined
    as, given a matrix :math:`A\in\mathbb{C}^{N\times N}` and a vector
    :math:`\vec{b}\in\mathbb{C}^{N}`, find :math:`\vec{x}\in\mathbb{C}^{N}` satisfying
    :math:`A\vec{x}=\vec{b}`.

    A system of linear equations is called :math:`s`-sparse if :math:`A` has at most :math:`s`
    non-zero entries per row or column. Solving an :math:`s`-sparse system of size :math:`N` with
    a classical computer requires :math:`\mathcal{ O }(Ns\kappa\log(1/\epsilon))` running time
    using the conjugate gradient method. Here :math:`\kappa` denotes the condition number of the
    system and :math:`\epsilon` the accuracy of the approximation.

    The HHL is a quantum algorithm to estimate a function of the solution with running time
    complexity of :math:`\mathcal{ O }(\log(N)s^{2}\kappa^{2}/\epsilon)` when
    :math:`A` is a Hermitian matrix under the assumptions of efficient oracles for loading the
    data, Hamiltonian simulation and computing a function of the solution. This is an exponential
    speed up in the size of the system, however one crucial remark to keep in mind is that the
    classical algorithm returns the full solution, while the HHL can only approximate functions of
    the solution vector.

    Examples:

        .. jupyter-execute::

            import numpy as np
            from qiskit import QuantumCircuit
            from quantum_linear_solvers.linear_solvers.hhl import HHL
            from quantum_linear_solvers.linear_solvers.matrices import TridiagonalToeplitz
            from quantum_linear_solvers.linear_solvers.observables import MatrixFunctional

            matrix = TridiagonalToeplitz(2, 1, 1 / 3, trotter_steps=2)
            right_hand_side = [1.0, -2.1, 3.2, -4.3]
            observable = MatrixFunctional(1, 1 / 2)
            rhs = right_hand_side / np.linalg.norm(right_hand_side)

            # Initial state circuit
            num_qubits = matrix.num_state_qubits
            qc = QuantumCircuit(num_qubits)
            qc.isometry(rhs, list(range(num_qubits)), None)

            hhl = HHL()
            solution = hhl.solve(matrix, qc, observable)
            approx_result = solution.observable

    References:

        [1]: Harrow, A. W., Hassidim, A., Lloyd, S. (2009).
        Quantum algorithm for linear systems of equations.
        `Phys. Rev. Lett. 103, 15 (2009), 1–15. <https://doi.org/10.1103/PhysRevLett.103.150502>`_

        [2]: Carrera Vazquez, A., Hiptmair, R., & Woerner, S. (2020).
        Enhancing the Quantum Linear Systems Algorithm using Richardson Extrapolation.
        `arXiv:2009.04484 <http://arxiv.org/abs/2009.04484>`_

    """

    def __init__(
        self,
        epsilon: float = 1e-2
    ) -> None:
        r"""
        Args:
            epsilon: Error tolerance of the approximation to the solution, i.e. if :math:`x` is the
                exact solution and :math:`\tilde{x}` the one calculated by the algorithm, then
                :math:`||x - \tilde{x}|| \le epsilon`.
            expectation: The expectation converter applied to the expectation values before
                evaluation. If None then PauliExpectation is used.
            quantum_instance: Quantum Instance or Backend. If None, a Statevector calculation is
                done.
        """
        super().__init__()

        self._epsilon = epsilon
        # Tolerance for the different parts of the algorithm as per [1]
        self._epsilon_r = epsilon / 3  # conditioned rotation
        self._epsilon_s = epsilon / 3  # state preparation
        self._epsilon_a = epsilon / 6  # hamiltonian simulation

        self._scaling = None  # scaling of the solution

        self._sampler = None

        # For now the default reciprocal implementation is exact
        self._exact_reciprocal = True
        # Set the default scaling to 1
        self.scaling = 1
        self.observable = None

    @property
    def scaling(self) -> float:
        """The scaling of the solution vector."""
        return self._scaling

    @scaling.setter
    def scaling(self, scaling: float) -> None:
        """Set the new scaling of the solution vector."""
        self._scaling = scaling

    def get_observable(self,qc):
        
        simulator = AerSimulator()
        circ = transpile(qc, simulator)

        # Run and get counts
        result = simulator.run(circ).result()
        counts = result.get_counts(circ)
        print(counts)


        self.observable = counts


    def _get_delta(self, n_l: int, lambda_min: float, lambda_max: float) -> float:
        """Calculates the scaling factor to represent exactly lambda_min on nl binary digits.

        Args:
            n_l: The number of qubits to represent the eigenvalues.
            lambda_min: the smallest eigenvalue.
            lambda_max: the largest eigenvalue.

        Returns:
            The value of the scaling factor.
        """
        formatstr = "#0" + str(n_l + 2) + "b"
        lambda_min_tilde = np.abs(lambda_min * (2**n_l - 1) / lambda_max)
        # floating point precision can cause problems
        if np.abs(lambda_min_tilde - 1) < 1e-7:
            lambda_min_tilde = 1
        binstr = format(int(lambda_min_tilde), formatstr)[2::]
        lamb_min_rep = 0
        for i, char in enumerate(binstr):
            lamb_min_rep += int(char) / (2 ** (i + 1))
        return lamb_min_rep


    def construct_circuit(
        self,
        matrix: np.ndarray,
        vector: np.ndarray
    ) -> QuantumCircuit:
        """Construct the HHL circuit.

        Args:
            matrix: The matrix specifying the system, i.e. A in Ax=b.
            vector: The vector specifying the right hand side of the equation in Ax=b.
            neg_vals: States whether the matrix has negative eigenvalues. If False the
            computation becomes cheaper.

        Returns:
            The HHL circuit.

        Raises:
            ValueError: If the input is not in the correct format.
            ValueError: If the type of the input matrix is not supported.
        """
        # State preparation circuit - default is qiskit
        nb = int(np.log2(len(vector)))
        vector_circuit = QuantumCircuit(nb)
            # pylint: disable=no-member
        vector_circuit.initialize(vector / np.linalg.norm(vector), list(range(nb)), None)

        # If state preparation is probabilistic the number of qubit flags should increase
        nf = 1

        # Hamiltonian simulation circuit
        matrix = np.array(matrix)  # Ensure the matrix has a compatible type
        U = expm(-1j * matrix * 2 * np.pi)  # Unitary matrix corresponding to e^(-i*A*2π)
        U_circuit = UnitaryGate(U)
        #matrix_circuit = NumPyMatrix(matrix, evolution_time=2 * np.pi)

        #---------------------------------------------------------------------------------------------------------------

        # check if the matrix can calculate the condition number and store the upper bound
        kappa = np.linalg.cond(U)

        # Update the number of qubits required to represent the eigenvalues
        # The +neg_vals is to register negative eigenvalues because
        # e^{-2 \pi i \lambda} = e^{2 \pi i (1 - \lambda)}
        nl = max(nb + 1, int(np.ceil(np.log2(kappa + 1)))) #+ neg_vals

        #--------------------------------------------------------------------------------------------------------------

        # check if the matrix can calculate bounds for the eigenvalues
        lambda_min, lambda_max =min(np.abs(np.linalg.eigvals(U))), max(np.abs(np.linalg.eigvals(U)))

        # Constant so that the minimum eigenvalue is represented exactly, since it contributes
        # the most to the solution of the system. -1 to take into account the sign qubit
        delta = self._get_delta(nl, lambda_min, lambda_max)
        # Update evolution time
        U_circuit.evolution_time = (
                2 * np.pi * delta / lambda_min #/ (2**neg_vals)
            )
        # Update the scaling of the solution
        self.scaling = lambda_min

        #-----------------------------------------------------------------------------------------------------------------

        reciprocal_circuit = ExactReciprocal(nl, delta)
        # Update number of ancilla qubits
        na = 0

        #-------------------------------------------------------------------------------------------------------------------
        
        # Initialise the quantum registers
        qb = QuantumRegister(nb)  # right hand side and solution
        ql = QuantumRegister(nl)  # eigenvalue evaluation qubits
        if na > 0:
            qa = AncillaRegister(na)  # ancilla qubits
        qf = QuantumRegister(nf)  # flag qubits
        cl = ClassicalRegister(nb)

        if na > 0:
            qc = QuantumCircuit(qb, ql, qa, qf,cl)
        else:
            qc = QuantumCircuit(qb, ql, qf,cl)

        # State preparation
        qc.append(vector_circuit, qb[:])
        # QPE
        phase_estimation = PhaseEstimation(nl, U_circuit)
        if na > 0:
            qc.append(
                phase_estimation, ql[:] + qb[:] + qa[: 0]
            )
        else:
            qc.append(phase_estimation, ql[:] + qb[:])
        # Conditioned rotation
        if self._exact_reciprocal:
            qc.append(reciprocal_circuit, ql[::-1] + [qf[0]])
        else:
            qc.append(
                reciprocal_circuit.to_instruction(),
                ql[:] + [qf[0]] + qa[: reciprocal_circuit.num_ancillas],
            )
        # QPE inverse
        if na > 0:
            qc.append(
                phase_estimation.inverse(),
                ql[:] + qb[:] + qa[: U_circuit.num_ancillas],
            )
        else:
            qc.append(phase_estimation.inverse(), ql[:] + qb[:])


        qc.measure(qb,cl) 
    
        #------------------------------------------------------------------------------------------------

        self.get_observable(qc)

        return qc

    def solve(
        self,
        matrix: np.ndarray,
        vector:  np.ndarray,
    ):
        """Tries to solve the given linear system of equations.

        Args:
            matrix: The matrix specifying the system, i.e. A in Ax=b.
            vector: The vector specifying the right hand side of the equation in Ax=b.
            observable: Optional information to be extracted from the solution.
                Default is the probability of success of the algorithm.
            observable_circuit: Optional circuit to be applied to the solution to extract
                information. Default is `None`.
            post_processing: Optional function to compute the value of the observable.
                Default is the raw value of measuring the observable.

        Raises:
            ValueError: If an invalid combination of observable, observable_circuit and
                post_processing is passed.

        Returns:
            The result object containing information about the solution vector of the linear
            system.
        """

        solution = HHL()
        solution.state = self.construct_circuit(matrix, vector)

        return solution

In [127]:
# Example usage
A = [[1,0,0,0],
     [0,1,0,0],
     [0,0,1,0],
     [0,0,0,1]]


b = [0, 0,0,1]

hhl = HHL()
solution = hhl.solve(A, b)

print(solution.state)

{'11': 1024}
        ┌───────────────┐┌──────┐        ┌─────────┐┌─┐   
q110_0: ┤0              ├┤3     ├────────┤3        ├┤M├───
        │  circuit-1767 ││      │        │         │└╥┘┌─┐
q110_1: ┤1              ├┤4     ├────────┤4        ├─╫─┤M├
        └───────────────┘│      │┌──────┐│         │ ║ └╥┘
q111_0: ─────────────────┤0 QPE ├┤2     ├┤0 QPE_dg ├─╫──╫─
                         │      ││      ││         │ ║  ║ 
q111_1: ─────────────────┤1     ├┤1     ├┤1        ├─╫──╫─
                         │      ││  1/x ││         │ ║  ║ 
q111_2: ─────────────────┤2     ├┤0     ├┤2        ├─╫──╫─
                         └──────┘│      │└─────────┘ ║  ║ 
  q112: ─────────────────────────┤3     ├────────────╫──╫─
                                 └──────┘            ║  ║ 
 c13: 2/═════════════════════════════════════════════╩══╩═
                                                     0  1 
